In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
diamond = pd.read_csv('diamonds.csv')

In [ ]:
diamond.head()

In [ ]:
diamond.info()

In [ ]:
diamond.describe()

In [ ]:
diamond.columns
diamond.drop(['Unnamed: 0'] , axis=1 , inplace=True)
diamond.describe()

In [ ]:
def converter(cut):
    if cut == "Ideal":
        return int(1)
    elif cut == "Premium":
        return int(2)
    elif cut == "Very Good":
        return int(3)
    elif cut == "Good":
        return int (4)
    elif cut == "Fair":
        return int(5) 
diamond["cut"] = diamond["cut"].apply(converter)
diamond.head()

In [ ]:
def converter(color):
    if color == "D":
        return int(1)
    elif color == "E":
        return int(2)
    elif color == "F":
        return int(3)
    elif color == "G":
        return int (4)
    elif color == "H":
        return int(5) 
    elif color == "I":
        return int(6) 
    elif color == "J":
        return int(7) 
diamond["color"] = diamond["color"].apply(converter)
diamond.head()

In [ ]:
def converter(clarity):
    if clarity == "IF":
        return int(1)
    elif clarity == "VVS1":
        return int(2)
    elif clarity == "VVS2":
        return int(3)
    elif clarity == "VS1":
        return int (4)
    elif clarity == "VS2":
        return int(5)
    elif clarity == "SI1":
        return int(6)    
    elif clarity == "SI2":
        return int(7) 
    elif clarity == "I1":
        return int(8)   
diamond["clarity"] = diamond["clarity"].apply(converter)
diamond.head()

In [ ]:
#sns.pairplot(diamond)

In [ ]:
#sns.distplot(diamond['price'])

In [ ]:
#sns.heatmap(diamond.corr())


In [ ]:
#sns.factorplot(data=diamond , kind='box' , size=7, aspect=2.5)

In [ ]:
#sns.kdeplot(diamond['carat'], shade=True , color='b')

In [ ]:
#sns.jointplot(x='carat' , y='price' , data=diamond , size=5)

In [ ]:
#sns.factorplot(x='cut', data=diamond , kind='count',aspect=2.5 )

In [ ]:
#sns.factorplot(x='cut', y='price', data=diamond, kind='box' ,aspect=2.5 )

In [ ]:
#sns.factorplot(x='color', data=diamond , kind='count',aspect=2.5 )

In [ ]:
#sns.factorplot(x='color', y='price' , data=diamond , kind='violin', aspect=2.5)

In [ ]:
#sns.boxplot(x='clarity', y='price', data=diamond)

In [ ]:
#plt.hist('depth' , data=diamond , bins=25)

In [ ]:
#sns.jointplot(x='depth', y='price' , data=diamond , kind='regplot', size=5)

In [ ]:
#sns.kdeplot(diamond['table'] ,shade=True , color='b')

In [ ]:
X = diamond[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
y = diamond['price']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=101)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression() 

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
coeff_df = pd.DataFrame(lr.coef_,X.columns,columns=['Coefficient'])
coeff_df

In [ ]:
predictions = lr.predict(X_test)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
sns.distplot((y_test-predictions),bins=50);

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
#Prozentualer Anteil einmal ausrechnen!
#print('MSE:', metrics.mean_squared_error(y_test, predictions))
#print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

#Neuronales Netz auf Basis von linearer Regression

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
diamond.isna().sum()

In [ ]:
#Nicht notwendig, da überall Werte vorhanden
#diamond = diamond.dropna()


In [ ]:
train_dataset = diamond.sample(frac=0.8,random_state=0)
test_dataset = diamond.drop(train_dataset.index)

In [ ]:
#Inspection of the data, 
sns.pairplot(train_dataset[["carat", "cut", "color", "clarity", "depth", "table", "x", "y", "z", "price"]], diag_kind="kde")

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("price")
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('price')
test_labels = test_dataset.pop('price')

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse', # We optimize for minimizing the mean squared error (mse)
                optimizer=optimizer,
                metrics=['mae', 'mse']) # We evaluate with the mean absolute error (mae) and the mean squared error (mse)
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

# EPOCHS = 1000
EPOCHS = 300 

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [price]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$price^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} price".format(mae))


In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [price]')
plt.ylabel('Predictions [price]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])


In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [price]")
_ = plt.ylabel("Count")